In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC


In [2]:
path = 'SDSS_DR16_90cutoff_for_Vicky.csv'
df = pd.read_csv(path)


In [3]:
rs = df.loc[:, 'z']
Types = df.loc[0:500, 'Types']
types = Types.apply(lambda v: v.replace(
    'elliptical', '1').replace('spiral', '0')).astype(int)
fd = df.loc[:, 'fracDeV_r']

X = np.array(df.loc[0:500, 'petroMag_u':'petroMag_z'])
# u, g, r, i, z band 0, 1, 2, 3, 4
ug = X[:, [0, 1]]
ur = X[:, [0, 2]]
ui = X[:, [0, 3]]
uz = X[:, [0, 4]]
gr = X[:, [1, 2]]
gi = X[:, [1, 3]]
gz = X[:, [1, 4]]
ri = X[:, [2, 3]]
rz = X[:, [2, 4]]
iz = X[:, [3, 4]]
y = np.array(types)

cmap = 'Paired'


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [5]:
C_list = range(1, 51, 1)
gamma_list = range(1, 11, 1)


# RBF Kernel loop version
def rbf_svc_loop(X_train, X_test, y_train, y_test, C_list, gamma_list):
    rbf_list = []

    for C in C_list:
        for gamma in gamma_list:
            clf_rbfSVC = Pipeline(
                [("rbf_svc", SVC(kernel="rbf", C=C, gamma=gamma, max_iter=10000))])

            clf_rbfSVC.fit(X_train, y_train)

            train = clf_rbfSVC.score(X_train, y_train)
            test = clf_rbfSVC.score(X_test, y_test)
            rbf_list += [(C, gamma, train, test)]

    return clf_rbfSVC, rbf_list


clf_rbfSVC, train_test_list = rbf_svc_loop(
    X_train, X_test, y_train, y_test, C_list, gamma_list)


In [6]:
# max test set
def max_test():
    POS = 3
    rbf_max = (0, 0, 0, 0)
    for test in train_test_list:
        if rbf_max[POS] < test[POS]:
            rbf_max = test

    C_max = rbf_max[0]
    gamma_max = rbf_max[1]
    train_max = rbf_max[2]
    test_max = rbf_max[3]

    return C_max, gamma_max, train_max, test_max


C, gamma, train, test = max_test()


In [7]:
print('C = ' + str(C) + ', ' + 'gamma = ' + str(gamma))
print("Train set accuracy = " + str(train))
print("Test set accuracy = " + str(test))

C = 44, gamma = 2
Train set accuracy = 0.8933333333333333
Test set accuracy = 0.8095238095238095
